# 제 1유형

In [16]:
import pandas as pd

iris = pd.read_csv('iris.csv')
SW = iris['Sepal.Width']
SW_mean = SW.mean()
SW_std = SW.std()
under_bound = SW_mean - 3*SW_std
over_bound = SW_mean + 3*SW_std
result_list = []
# print(under_bound, over_bound)

print(SW[(SW <= under_bound) | (SW >= over_bound)].sum())

4.4


In [79]:
import pandas as pd

mtcars = pd.read_csv('mtcars1.csv')
disp = mtcars['disp']
# print(disp)

sorted_mtcars = mtcars.sort_values(by='disp', ascending=False)
rank_series = sorted_mtcars['disp'].rank(ascending=False, method='min')

sorted_mtcars['rank'] = rank_series

print(round(sorted_mtcars[sorted_mtcars['rank'] <= 20]['disp'].std(), 2))

97.47


In [89]:
import pandas as pd

cars = pd.read_csv('Cars93.csv')

# print(cars.info())

all_record = len(cars)
# print(all_record)

na_record = cars.isna().sum()
na_col_record = na_record[na_record >= 1].count()
# print(na_col_record)

all_na_record = na_record.sum()
# print(all_na_record)

remove_na_record = len(cars[na_record[na_record >= 10].index].dropna())
# print(remove_na_record)

two_na_record = sum(cars[cars.isna().sum(axis=1) >= 2].index)
# print(two_na_record)

print(all_record + na_col_record + all_na_record + remove_na_record + two_na_record)

1090


# 제 2유형

In [92]:
import pandas as pd

X_train = pd.read_csv('titanic3_X_train.csv')
X_test = pd.read_csv('titanic3_X_test.csv')
y_train = pd.read_csv('titanic3_y_train.csv')
y_test = pd.read_csv('titanic3_y_test.csv')

ID = X_test['ID']

y_train = y_train['survived']
y_test = y_test['survived']

# print(X_train)
# print(y_test)

# print(X_train.info()) # age, fare, cabin, embarked 결측치 존재
# print(X_test.info()) # age, cabin, embarked 결측치 존재

# ID 컬럼 제거
X_train.drop('ID', axis=1, inplace=True)
X_test.drop('ID', axis=1, inplace=True)

# print(X_train.info())

# pclass (문제 없음)
# print(X_train['pclass'].value_counts())
# print(X_test['pclass'].value_counts())

# name 컬럼 제거
X_train.drop('name', axis=1, inplace=True)
X_test.drop('name', axis=1, inplace=True)

# print(X_train.info())

# sex (F를 female로 변환)
# print(X_train['sex'].value_counts())
# print(X_test['sex'].value_counts())

X_train['sex'] = X_train['sex'].replace('F', 'female').map({'female' : 0, 'male' : 1})
X_test['sex'] = X_test['sex'].replace('F', 'female').map({'female' : 0, 'male' : 1})

# print(X_train['sex'].value_counts())
# print(X_test['sex'].value_counts())

# print(X_train.info())

# print(X_train['age'].isna().sum())

# age 성별에 따른 나이 평균 결측치 대체
X_train['age'] = X_train['age'].fillna(X_train.groupby('sex')['age'].transform('mean'))
X_test['age'] = X_test['age'].fillna(X_test.groupby('sex')['age'].transform('mean'))

# print(X_train.info())

# fare 컬럼 평균으로 결측치 대체
fare_mean_train = X_train['fare'].mean()
X_train['fare'] = X_train['fare'].fillna(fare_mean_train)
# print(X_train.info())

# cabin 컬럼 제거
X_train.drop('cabin', axis=1, inplace=True)
X_test.drop('cabin', axis=1, inplace=True)
# print(X_train.info())

# sibsp 컬럼 (문제 없음)
# print(X_train['sibsp'].value_counts())
# print(X_test['sibsp'].value_counts())

# parch 컬럼 (문제 없음)
# print(X_train['parch'].value_counts())
# print(X_test['parch'].value_counts())

# ticket 컬럼 (의미 불명이라 제거하기로 결정)
# print(X_train['ticket'].value_counts())
# print(X_test['ticket'].value_counts())
X_train.drop('ticket', axis=1, inplace=True)
X_test.drop('ticket', axis=1, inplace=True)

# print(X_train.info())

# embarked 컬럼 (최빈값으로 결측치 대체)
# print(X_train['embarked'].value_counts())
# print(X_test['embarked'].value_counts())
X_train['embarked'] = X_train['embarked'].fillna(X_train['embarked'].mode()[0])
X_test['embarked'] = X_test['embarked'].fillna(X_test['embarked'].mode()[0])

embarked_OH_train = pd.get_dummies(X_train['embarked'])
embarked_OH_test = pd.get_dummies(X_test['embarked'])

# print(embarked_OH_train)

X_train = pd.concat([X_train, embarked_OH_train], axis=1).drop('embarked', axis=1)
X_test = pd.concat([X_test, embarked_OH_test], axis=1).drop('embarked', axis=1)

# print(X_train.info())
# print(X_test.info())

from sklearn.model_selection import train_test_split
X_trainDF, X_validDF, y_trainSR, y_validSR = train_test_split(X_train, y_train,
                                                              train_size = 0.8,
                                                              stratify=y_train, random_state=42)
print(X_trainDF.shape, X_validDF.shape, y_trainSR.shape, y_validSR.shape)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
rf = RandomForestClassifier(random_state=1)
xgb = XGBClassifier(random_state=1)
lgb = LGBMClassifier(random_state=1)

rf.fit(X_trainDF, y_trainSR)
xgb.fit(X_trainDF, y_trainSR)
lgb.fit(X_trainDF, y_trainSR)

pred_y_rf = rf.predict(X_validDF)
pred_y_xgb = xgb.predict(X_validDF)
pred_y_lgb = lgb.predict(X_validDF)

from sklearn.metrics import *
f1_rf = f1_score(y_validSR, pred_y_rf)
f1_xgb = f1_score(y_validSR, pred_y_xgb)
f1_lgb = f1_score(y_validSR, pred_y_lgb)

print(f1_rf, f1_xgb, f1_lgb)

pred_y_test_rf = rf.predict(X_test)
f1_rf_test = f1_score(y_test, pred_y_test_rf)
print(f1_rf_test)

output = pd.DataFrame({
    'ID' : ID,
    'survived' : pred_y_test_rf
})

print(output)

output.to_csv('모의고사1회.csv', index=False)

(628, 9) (157, 9) (628,) (157,)
0.736 0.7301587301587301 0.7401574803149606
0.7211538461538461
       ID  survived
0     786         1
1     787         0
2     788         0
3     789         0
4     790         0
..    ...       ...
519  1305         0
520  1306         1
521  1307         1
522  1308         0
523  1309         1

[524 rows x 2 columns]


# 제 3유형

In [108]:
import pandas as pd
from scipy.stats import *

cars = pd.read_csv('Cars93.csv')
# print(cars)

small = cars[cars['Type'] == 'Small']['Price'].dropna()
large = cars[cars['Type'] == 'Large']['Price'].dropna()

mu1 = large.mean()
mu2 = small.mean()

# (a)
print(round(mu1 - mu2, 2))


stat, pvalue = ttest_ind(large, small, equal_var=True)

# stat = (mu1 - mu2) / se
se = (mu1 - mu2) / stat

# (b)
print(round(se, 2))

dof = len(small) + len(large) - 2
lower_cl = round(t.interval(0.95, dof, mu1-mu2, se)[0], 1)
print(lower_cl)
print('기각')

13.85
1.61
10.5
기각


In [84]:
import pandas as pd
import math as m
from scipy.stats import *

dices = pd.read_csv('dices.csv')
# print(dices)

dice1 = dices[dices['scale'] == 1]
dice2 = dices[dices['scale'] == 2]
dice3 = dices[dices['scale'] == 3]
dice4 = dices[dices['scale'] == 4]
dice5 = dices[dices['scale'] == 5]
dice6 = dices[dices['scale'] == 6]

observed_freq = [len(dices[dices['scale'] == i]) for i in range(1, 7)]
# print(observed_freq)

expected_ratio = [1/6] * 6
expected_freq = [i*len(dices) for i in expected_ratio]
# print(expected_freq)

# (a)
print(m.floor(expected_freq[2]))


stat, pvalue = chisquare(observed_freq, expected_freq)
# (b)
print(m.floor(stat))

# (c)
print(round(pvalue, 4) if round(pvalue, 4) >= 0.0001 else 0)
print('기각')

83
411
0
기각
